In [9]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [10]:
import subprocess
#This line imports the subprocess module, which allows us to run shell commands (e.g., compile and execute C programs) from within Python. This is crucial for interacting with external systems like compilers.

In [11]:
#!pip install -U nltk

In [12]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [13]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [14]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [21]:
system_message = "You are a code translator, you translate from c to Java and  the output should only be code"

In [69]:
system_message_3 = "You are a code translator, you translate from c to c++ and  the output should only be code"

In [91]:
system_message_2 = "You are a code translator, you translate from c to c++ and  the output should only be code , remove any extra english text or any extra punctation mark it needs to be a ready to run code. so remove apostrophe  marks "

In [234]:
# Store the C code in a variable
original = """#include <stdio.h>

int main() {
    int a=2, b=3, sum = 0;
    
    printf("Enter two integers: ");
    scanf("%d %d", &a, &b);

    sum = a + b;

    printf("Sum: %d", sum);

    return 0;
}
"""

# Use the variable in the prompt message
prompt_2 = f"Translate following C code to C++:\n{original}"


In [236]:
 messages = [
        {"role": "system", "content": system_message_2},
        {"role": "user", "content": prompt_2}
      ]

In [238]:
response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )

print ( response.choices[0].message.content )

#include <iostream>
using namespace std;

int main() {
    int a = 2, b = 3, sum = 0;

    cout << "Enter two integers: ";
    cin >> a >> b;

    sum = a + b;

    cout << "Sum: " << sum;

    return 0;
}


In [239]:
generated = response.choices[0].message.content.strip()
print (generated)



#include <iostream>
using namespace std;

int main() {
    int a = 2, b = 3, sum = 0;

    cout << "Enter two integers: ";
    cin >> a >> b;

    sum = a + b;

    cout << "Sum: " << sum;

    return 0;
}


In [196]:
refrence = """
#include <iostream>

int main() {
    int a, b, sum = 0;

    std::cout << "Enter two integers: ";
    std::cin >> a >> b;

    sum = a + b;

    std::cout << "Sum: " << sum;

    return 0;
}

"""

In [198]:
refrence_tokens = refrence.split()
generated_tokens = generated.split()

In [200]:
print (generated_tokens)
print (refrence_tokens)

['#include', '<iostream>', 'using', 'namespace', 'std;', 'int', 'main()', '{', 'int', 'a', '=', '2,', 'b', '=', '3,', 'sum', '=', '0;', 'cout', '<<', '"Enter', 'two', 'integers:', '";', 'cin', '>>', 'a', '>>', 'b;', 'sum', '=', 'a', '+', 'b;', 'cout', '<<', '"Sum:', '"', '<<', 'sum', '<<', 'endl;', 'return', '0;', '}']
['#include', '<iostream>', 'int', 'main()', '{', 'int', 'a,', 'b,', 'sum', '=', '0;', 'std::cout', '<<', '"Enter', 'two', 'integers:', '";', 'std::cin', '>>', 'a', '>>', 'b;', 'sum', '=', 'a', '+', 'b;', 'std::cout', '<<', '"Sum:', '"', '<<', 'sum;', 'return', '0;', '}']


In [202]:
smooth_fn = SmoothingFunction().method4

In [204]:
bleu_score =  sentence_bleu (refrence_tokens, generated_tokens, smoothing_function = smooth_fn)

In [206]:
# pip install --pre nltk

In [210]:
print ( " code Bleu Score " )
print ( bleu_score)


 code Bleu Score 
0.010778654685071103


In [214]:
# Metric: Match Success Rate (MSR) Function 
def compute_msr(refrence_tokens, generated_tokens):
    return refrence_tokens == generated_tokens


msr_score = compute_msr(refrence_tokens, generated_tokens)

print(f"Match Success Rate (MSR): {msr_score}")


Match Success Rate (MSR): False


In [216]:
generated = response.choices[0].message.content.strip()
print (generated)
print (refrence)


#include <iostream>
using namespace std;

int main() {
    int a = 2, b = 3, sum = 0;

    cout << "Enter two integers: ";
    cin >> a >> b;

    sum = a + b;

    cout << "Sum: " << sum << endl;

    return 0;
}

#include <iostream>

int main() {
    int a, b, sum = 0;

    std::cout << "Enter two integers: ";
    std::cin >> a >> b;

    sum = a + b;

    std::cout << "Sum: " << sum;

    return 0;
}




In [254]:

# """
# This is an example that is not related to LLM to confirm evaluation metrices work
# """
# import subprocess
# import os

# # Function to execute C++ code and capture its output using gcc
# def execute_cpp_file_with_gcc(file_name):
#     """
#     Compile and execute an existing C++ file using gcc and return its output.
    
#     Parameters:
#     file_name (str): The name of the C++ file to compile and execute.
    
#     Returns:
#     str: The output of the compiled C++ program (stdout).
#     None: If there is an error during compilation or execution.
#     """
#     # Ensure output file name is valid (replace .cpp with a valid executable name)
#     output_file = file_name.replace('.cpp', '')

#     print(f"Compiling {file_name} to create {output_file}...")

#     # Compile the C++ code using gcc (with -x c++ to specify C++ code) and create an executable
#     compile_process = subprocess.run(
#         ['gcc', '-x', 'c++', file_name, '-o', output_file, '-lstdc++'], 
#         capture_output=True, text=True
#     )
    
#     # Check if the compilation was successful
#     if compile_process.returncode != 0:
#         print(f"Compilation Error for {file_name}: {compile_process.stderr}")
#         return None  # Return None if compilation failed
    
#     print(f"Compilation successful, executable created: {output_file}")
    
#     # Run the compiled executable and capture its output (stdout)
#     run_process = subprocess.run([f'./{output_file}'], capture_output=True, text=True)
    
#     if run_process.returncode != 0:
#         print(f"Execution Error: {run_process.stderr}")
#         return None  # If execution fails, return None
    
#     print(f"Execution Output:\n{run_process.stdout.strip()}")
#     return run_process.stdout.strip()

# # Function to check Computational Accuracy (CA)
# def compute_computational_accuracy(reference_file, generated_file):
#     """
#     Compare the outputs of two C++ programs (saved as files) for computational accuracy.

#     Parameters:
#     reference_file (str): The path to the reference C++ file.
#     generated_file (str): The path to the generated C++ file.

#     Returns:
#     bool: True if the outputs of both programs match, False otherwise.
#     """
    
#     # Save the reference C++ code to a .cpp file (for viewing before compilation)
#     with open("reference_code.cpp", "w") as file:
#         file.write(reference_file)

#     # Save the generated C++ code to a .cpp file (for viewing before compilation)
#     with open("generated_code.cpp", "w") as file:
#         file.write(generated_file)

#     # Inform the user that the C++ code has been saved to files
#     print("C++ code has been saved to reference_code.cpp and generated_code.cpp")
    
#     # Execute the reference C++ code and the generated C++ code to capture their outputs
#     reference_output = execute_cpp_file_with_gcc("reference_code.cpp")
#     generated_output = execute_cpp_file_with_gcc("generated_code.cpp")
    
#     # If either of the outputs is None (due to error), return False
#     if reference_output is None or generated_output is None:
#         return False
    
#     # Clean up the generated C++ files after execution
#     # os.remove("reference_code.cpp")
#     # os.remove("generated_code.cpp")
    
#     # Compare the outputs and return True if they match, False otherwise
#     return reference_output == generated_output

# # Example reference C++ code (C to C++ translation of the C example)
# reference_cpp = """
# #include <iostream>
# using namespace std;

# int main() {
#     int a, b, sum;
#     a=2;
#     b=3;
   
    
#     sum = a + b;
    
#     cout << "Sum: " << sum << endl;
    
#     return 0;
# }
# """

# # Example generated C++ code (could be the result of a transformation process)
# generated_cpp = """
# #include <iostream>
# using namespace std;

# int main() {
#     int a, b, c, sum;
#     a=2;
#     b=3;
#     c=7;
#     sum=0;
    
#     sum = a + b;
    
#     cout << "Sum: " << sum << endl;
    
#     return 0;
# }
# """



# # Example reference C++ code (C to C++ translation of the C example)
# greference_cpp = """
# #include <iostream>
# using namespace std;

# int main() {
#     int a, b, sum;
    
#     cout << "Enter two integers: ";
#     cin >> a >> b;
    
#     sum = a + b;
    
#     cout << "Sum: " << sum << endl;
    
#     return 0;
# }
# """
# # Example generated C++ code (could be the result of a transformation process)
# ggenerated_cpp = """
# #include <iostream>
# using namespace std;

# int main() {
#     int a, b, sum;
    
#     cout << "Enter two integers: ";
#     cin >> a >> b;
    
#     sum = a + b;
    
#     cout << "Sum: " << sum << endl;
    
#     return 0;
# }
# """


# # Check Computational Accuracy (CA)
# result = compute_computational_accuracy(reference_cpp, generated_cpp)

# # Print the result of the comparison
# print("Computational Accuracy:", result)  # Output: True if outputs match, False otherwise


In [252]:
# """
# This is an example for c and c++ being compared for the CA it is not related to LLMS

# """
# import subprocess
# import os

# # Function to compile and execute C or C++ code using gcc
# def execute_file(file_name, is_cpp=False, executable_name=None):
#     """
#     Compile and execute a C or C++ file using gcc and return its output and compilation success.
    
#     Parameters:
#     file_name (str): The name of the file to compile and execute.
#     is_cpp (bool): True if it's a C++ file, False if it's a C file.
#     executable_name (str): The desired name for the compiled executable.
    
#     Returns:
#     tuple: (bool, str) where:
#         - bool: Compilation success (True/False)
#         - str: Program output (stdout) or error message (None if compilation/execution fails).
#     """
#     # Set the default name for the output executable if not provided
#     if not executable_name:
#         executable_name = "new_" + file_name.replace('.c', '').replace('.cpp', '') + '_executable'

#     print(f"Compiling {file_name} to create {executable_name}...")

#     # Use gcc for both C and C++ code
#     compile_command = ['gcc', file_name, '-o', executable_name]

#     # Add the -x c++ flag if compiling C++ code
#     if is_cpp:
#         compile_command.insert(1, '-x')
#         compile_command.insert(2, 'c++')
#         # Add the C++ standard library link flag
#         compile_command.append('-lstdc++')

#     # Compile the code using gcc
#     compile_process = subprocess.run(
#         compile_command, capture_output=True, text=True
#     )
    
#     # Check if the compilation was successful
#     if compile_process.returncode != 0:
#         print(f"Compilation Error for {file_name}: {compile_process.stderr}")
#         return False, None  # Compilation failed
    
#     print(f"Compilation successful, executable created: {executable_name}")
    
#     # Run the compiled executable and capture its output
#     run_process = subprocess.run([f'./{executable_name}'], capture_output=True, text=True)
    
#     if run_process.returncode != 0:
#         print(f"Execution Error: {run_process.stderr}")
#         return True, None  # Execution failed, but compilation succeeded
    
#     print(f"Execution Output:\n{run_process.stdout.strip()}")
#     return True, run_process.stdout.strip()  # Compilation and execution successful

# # Function to check Compilation Rate (CR) and Computational Accuracy (CA)
# def compute_cr_and_ca(reference_file, generated_file):
#     """
#     Compute the Compilation Rate (CR) for the generated C++ code and the Computational Accuracy (CA) by comparing
#     the outputs of the reference (C) and generated (C++) code
    
#     Parameters:
#     reference_file (str): The reference C code as a string.
#     generated_file (str): The generated C++ code as a string.
    
#     Returns:
#     dict: A dictionary with the metrics:
#         - Compilation Rate (CR)
#         - Computational Accuracy (CA)
#     """
#     # Save the reference and generated code to files
#     with open("reference_code.c", "w") as file:
#         file.write(reference_file)
#     with open("generated_code.cpp", "w") as file:
#         file.write(generated_file)
    
#     print("C code has been saved to reference_code.c and C++ code to generated_code.cpp")
    
#     # Execute the reference (C code) and generated (C++ code) files
#     ref_compiled, reference_output = execute_file("reference_code.c", is_cpp=False, executable_name="new_reference_code_executable")
#     gen_compiled, generated_output = execute_file("generated_code.cpp", is_cpp=True, executable_name="new_generated_code_executable")
    
#     # Compilation Rate (CR): Both files must compile successfully
#     compilation_rate = gen_compiled and ref_compiled 
    
#     # Computational Accuracy (CA): Both files must execute and their outputs must match
#     if reference_output is None or generated_output is None:
#         computational_accuracy = False
#     else:
#         computational_accuracy = reference_output == generated_output
    
#     # Return metrics as a dictionary
#     return {
#         "Compilation Rate (CR)": compilation_rate,
#         "Computational Accuracy (CA)": computational_accuracy
#     }

# # Example reference and generated code (replace with actual code)
# reference = """
# #include <stdio.h>

# int main() {
#     printf("Hello from C\\n");
#     return 0;
# }
# """

# generated = """
# #include <iostream>

# int main() {
#     std::cout << "Hello from C++" << std::endl;
#     return 0;
# }
# """

# # Compute Compilation Rate (CR) and Computational Accuracy (CA)
# metrics = compute_cr_and_ca(reference, generated)

# # Print the metrics
# for metric, score in metrics.items():
#     print(f"{metric}: {score}")


In [221]:
#Test using GPT
#Transalate C to C++

In [250]:

"""
This is comparing Refrence C++ and Translated C++
"""

import subprocess
import os

# Function to compile and execute C++ code
def execute_cpp_file_with_gcc(file_name):
    """
    Compile and execute a C++ file using gcc and return its output and compilation success.
    
    Parameters:
    file_name (str): The name of the C++ file to compile and execute.
    
    Returns:
    tuple: (bool, str) where:
        - bool: Compilation success (True/False)
        - str: Program output (stdout) or error message (None if compilation/execution fails).
    """
    output_file = file_name.replace('.cpp', '')

    print(f"Compiling {file_name} to create {output_file}...")

    # Compile the C++ code using gcc
    compile_process = subprocess.run(
        ['gcc', '-x', 'c++', file_name, '-o', output_file, '-lstdc++'], 
        capture_output=True, text=True
    )
    
    # Check if the compilation was successful
    if compile_process.returncode != 0:
        print(f"Compilation Error for {file_name}: {compile_process.stderr}")
        return False, None  # Compilation failed
    
    print(f"Compilation successful, executable created: {output_file}")
    
    # Run the compiled executable and capture its output
    run_process = subprocess.run([f'./{output_file}'], capture_output=True, text=True)
    
    if run_process.returncode != 0:
        print(f"Execution Error: {run_process.stderr}")
        return True, None  # Execution failed, but compilation succeeded
    
    print(f"Execution Output:\n{run_process.stdout.strip()}")
    return True, run_process.stdout.strip()  # Compilation and execution successful

# Function to check Compilation Rate (CR) and Computational Accuracy (CA)
def compute_cr_and_ca(reference_file, generated_file):
    """
   Compute the Compilation Rate (CR) for the generated code and the Computational Accuracy (CA) by comparing
    the outputs of the reference and generated code
    
    Parameters:
    reference_file (str): The reference C++ code as a string.
    generated_file (str): The generated C++ code as a string.
    
    Returns:
    dict: A dictionary with the metrics:
        - Compilation Rate (CR)
        - Computational Accuracy (CA)
    """
    # Save the reference and generated C++ code to files
    with open("reference_code.cpp", "w") as file:
        file.write(reference_file)
    with open("generated_code.cpp", "w") as file:
        file.write(generated_file)
    
    print("C++ code has been saved to reference_code.cpp and generated_code.cpp")
    
    # Execute the reference and generated C++ files
    ref_compiled, reference_output = execute_cpp_file_with_gcc("reference_code.cpp")
    gen_compiled, generated_output = execute_cpp_file_with_gcc("generated_code.cpp")
    
    # Compilation Rate (CR): Both files must compile successfully
    compilation_rate = gen_compiled 
    
    # Computational Accuracy (CA): Both files must execute and their outputs must match
    if reference_output is None or generated_output is None:
        computational_accuracy = False
    else:
        computational_accuracy = reference_output == generated_output
    
    # Return metrics as a dictionary
    return {
        "Compilation Rate (CR)": compilation_rate,
        "Computational Accuracy (CA)": computational_accuracy
    }


# Compute Compilation Rate (CR) and Computational Accuracy (CA)
metrics = compute_cr_and_ca(refrence, generated)

# Print the metrics
for metric, score in metrics.items():
    print(f"{metric}: {score}")

C++ code has been saved to reference_code.cpp and generated_code.cpp
Compiling reference_code.cpp to create reference_code...
Compilation successful, executable created: reference_code
Execution Output:
Enter two integers: Sum: 63733
Compiling generated_code.cpp to create generated_code...
Compilation successful, executable created: generated_code
Execution Output:
Enter two integers: Sum: 5
Compilation Rate (CR): True
Computational Accuracy (CA): False


In [256]:

"""
This is comparing Original C and translated C++
"""
import subprocess
import os

# Function to compile and execute C or C++ code using gcc
def execute_file(file_name, is_cpp=False, executable_name=None):
    """
    Compile and execute a C or C++ file using gcc and return its output and compilation success.
    
    Parameters:
    file_name (str): The name of the file to compile and execute.
    is_cpp (bool): True if it's a C++ file, False if it's a C file.
    executable_name (str): The desired name for the compiled executable.
    
    Returns:
    tuple: (bool, str) where:
        - bool: Compilation success (True/False)
        - str: Program output (stdout) or error message (None if compilation/execution fails).
    """
    # Set the default name for the output executable if not provided
    if not executable_name:
        executable_name = "new_" + file_name.replace('.c', '').replace('.cpp', '') + '_executable'

    print(f"Compiling {file_name} to create {executable_name}...")

    # Use gcc for both C and C++ code
    compile_command = ['gcc', file_name, '-o', executable_name]

    # Add the -x c++ flag if compiling C++ code
    if is_cpp:
        compile_command.insert(1, '-x')
        compile_command.insert(2, 'c++')
        # Add the C++ standard library link flag
        compile_command.append('-lstdc++')

    # Compile the code using gcc
    compile_process = subprocess.run(
        compile_command, capture_output=True, text=True
    )
    
    # Check if the compilation was successful
    if compile_process.returncode != 0:
        print(f"Compilation Error for {file_name}: {compile_process.stderr}")
        return False, None  # Compilation failed
    
    print(f"Compilation successful, executable created: {executable_name}")
    
    # Run the compiled executable and capture its output
    run_process = subprocess.run([f'./{executable_name}'], capture_output=True, text=True)
    
    if run_process.returncode != 0:
        print(f"Execution Error: {run_process.stderr}")
        return True, None  # Execution failed, but compilation succeeded
    
    print(f"Execution Output:\n{run_process.stdout.strip()}")
    return True, run_process.stdout.strip()  # Compilation and execution successful

# Function to check Compilation Rate (CR) and Computational Accuracy (CA)
def compute_cr_and_ca(reference_file, generated_file):
    """
    Compute the Compilation Rate (CR) for the generated C++ code and the Computational Accuracy (CA) by comparing
    the outputs of the reference (C) and generated (C++) code
    
    Parameters:
    reference_file (str): The reference C code as a string.
    generated_file (str): The generated C++ code as a string.
    
    Returns:
    dict: A dictionary with the metrics:
        - Compilation Rate (CR)
        - Computational Accuracy (CA)
    """
    # Save the reference and generated code to files
    with open("reference_code.c", "w") as file:
        file.write(reference_file)
    with open("generated_code.cpp", "w") as file:
        file.write(generated_file)
    
    print("C code has been saved to reference_code.c and C++ code to generated_code.cpp")
    
    # Execute the reference (C code) and generated (C++ code) files
    ref_compiled, reference_output = execute_file("reference_code.c", is_cpp=False, executable_name="new_reference_code_executable")
    gen_compiled, generated_output = execute_file("generated_code.cpp", is_cpp=True, executable_name="new_generated_code_executable")
    
    # Compilation Rate (CR): Both files must compile successfully
    compilation_rate = gen_compiled  
    
    # Computational Accuracy (CA): Both files must execute and their outputs must match
    if reference_output is None or generated_output is None:
        computational_accuracy = False
    else:
        computational_accuracy = reference_output == generated_output
    
    # Return metrics as a dictionary
    return {
        "Compilation Rate (CR)": compilation_rate,
        "Computational Accuracy (CA)": computational_accuracy
    }


# Compute Compilation Rate (CR) and Computational Accuracy (CA)
metrics = compute_cr_and_ca(original, generated)

# Print the metrics
for metric, score in metrics.items():
    print(f"{metric}: {score}")


C code has been saved to reference_code.c and C++ code to generated_code.cpp
Compiling reference_code.c to create new_reference_code_executable...
Compilation successful, executable created: new_reference_code_executable
Execution Output:
Enter two integers: Sum: 5
Compiling generated_code.cpp to create new_generated_code_executable...
Compilation successful, executable created: new_generated_code_executable
Execution Output:
Enter two integers: Sum: 5
Compilation Rate (CR): True
Computational Accuracy (CA): True
